In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('./dtl'))
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import math
from node import Node
from myC45 import Tree as tree_myC45
from myID3 import Tree as tree_myID3

In [2]:
# read data tennis
print('Data Tennis')
data_tennis = pd.read_csv("./data/play_tennis.csv")
print(data_tennis.head())

#make tennis tree
data_tennis = data_tennis.drop('day', axis=1)
tree_tennis = tree_myID3(data_tennis, 'play', use_info_gain=True)
root_tennis = tree_tennis.make_tree()
tree_tennis.print_tree(root_tennis, 0, 2)

Data Tennis
  day   outlook  temp humidity    wind play
0  D1     Sunny   Hot     High    Weak   No
1  D2     Sunny   Hot     High  Strong   No
2  D3  Overcast   Hot     High    Weak  Yes
3  D4      Rain  Mild     High    Weak  Yes
4  D5      Rain  Cool   Normal    Weak  Yes
-------tree-------
outlook
|--(Sunny)-->humidity
|            |--(High)-->{class : No}
|            |--(Normal)-->{class : Yes}
|--(Overcast)-->{class : Yes}
|--(Rain)-->wind
|            |--(Weak)-->{class : Yes}
|            |--(Strong)-->{class : No}


In [3]:
#read data iris
load, target = load_iris(return_X_y=True)
iris_data = pd.DataFrame(load, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
iris_data['label'] = pd.Series(target)
print(iris_data.head())

#make iris tree
tree_iris = tree_myC45(iris_data, 'label')
root_iris = tree_iris.make_tree()
tree_iris.print_tree(root_iris, 0, 2)

   sepal_length  sepal_width  petal_length  petal_width  label
0           5.1          3.5           1.4          0.2      0
1           4.9          3.0           1.4          0.2      0
2           4.7          3.2           1.3          0.2      0
3           4.6          3.1           1.5          0.2      0
4           5.0          3.6           1.4          0.2      0
-------tree-------
petal_length
|--(<=2.45)-->{class : 0}
|--(>2.45)-->petal_width
|            |--(<=1.7)-->sepal_length
|            |            |--(<=7.1)-->sepal_width
|            |            |            |--(<=2.8)-->{class : 1}
|            |            |            |--(>2.8)-->{class : 1}
|            |            |--(>7.1)-->{class : 2}
|            |--(>1.7)-->sepal_length
|            |            |--(<=5.9)-->sepal_width
|            |            |            |--(<=3.1)-->{class : 2}
|            |            |            |--(>3.1)-->{class : 1}
|            |            |--(>5.9)-->{class : 2}


In [26]:
def get_model_myID3(node, depth, space, array):
    if (depth == 0):
        dash = ''
    else:
        dash = ('--'*space)*depth + '>' + node.parent_value + ' '
        
    if(node.is_leaf):
        output = dash + '{' + str(node.leaf_value) + '}'
    else:
        output = dash + '[' + node.split_attr + ']' 
    
    # print(output)
    array.append(output)
    
    depth += 1
    for child in node.childs:
        get_model_myID3(child, depth, space, array)

In [27]:
def get_model_myC45(node, depth, space, array):
    if (depth == 0):
        dash = ''
    else:
        dash = ('--'*space)*depth + node.parent_value + ' '
        
    if(node.is_leaf):
        output = dash + '{' + str(node.leaf_value) + '}'
    else:
        output = dash + '[' + node.split_attr + ']' 
    
    # print(output)
    array.append(output)
    
    depth += 1
    for child in node.childs:
        get_model_myC45(child, depth, space, array)

In [34]:
# export file
def export_tree(filename, tipe):
    model = []
    if(tipe=="ID3"):
        get_model_myID3(root_tennis, 0, 2, model)
    elif (tipe=="C45"):
        get_model_myC45(root_iris, 0, 2, model)
        
    text_file = open(filename, "w")
    for L in model:
        text_file.write(L+"\n")
    text_file.close()

In [35]:
#exporting file
export_tree("tree_ID3.txt", "ID3")
export_tree("tree_C45.txt", "C45")